In [7]:
#Libraries
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv() 

True

In [8]:
#API's
api_key = os.getenv("OPENAI_API_KEY")


# LLM Chain

In [9]:
#LLM
llm = ChatOpenAI(openai_api_key=api_key)  # Create the ChatGPT instanc
llm.invoke("who is Valentin Valencia?")



AIMessage(content='There is no widely known or notable figure named Valentin Valencia. It is possible that Valentin Valencia is a private individual or a fictional character.')

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class personal assistant."),
    ("user", "{input}")
])

In [11]:
chain = prompt | llm 

In [13]:
chain.invoke({"input": "who is Valentin Valencia?"})

AIMessage(content="I'm sorry, but I couldn't find any specific information on a person named Valentin Valencia. It is possible that this individual is not widely known or does not have a public presence. If you have more context or details, I may be able to provide more relevant information.")

In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [15]:
chain = prompt | llm | output_parser

In [16]:
chain.invoke({"input": "who is Valentin Valencia?"})

"I'm sorry, but I couldn't find any specific information about a person named Valentin Valencia. Could you provide more context or details to help me assist you better?"

# Retrieval Chain

In [17]:
"""
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/President_of_Colombia")

docs = loader.load()
"""
# Load content from the local file
file_path = "text.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_content = file.read()

In [18]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [ ]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents([Document(page_content=text_content)])
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:


document_chain.invoke({
    "input": "who is Valentin Valencia?",
    "context": [Document(page_content="the president of Colombia is Rodolfo Hernández, born in New York")]
})

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "who is Valentin Valencia?"})
print(response["answer"])